Extract tickers obtaineed from ssm.ipynb

In [2]:
from pprint import pprint

ticker_names = []

with open(r'./subset_c.txt', 'r') as fp:
    for line in fp:
        x = line[:-1] # excluding \n char
        ticker_names.append(x)

pprint(ticker_names)

['ESNT',
 'ATO',
 'VLO',
 'PCH',
 'WSO',
 'REG',
 'BX',
 'CDP',
 'DLB',
 'ITIC',
 'MATX',
 'FOXA',
 'CEIX',
 'SJM',
 'MTH',
 'KDP',
 'INTC',
 'PEAK',
 'MGY',
 'CHRD',
 'MLM',
 'AMH',
 'GIL',
 'OC',
 'IEX',
 'KMI',
 'SITC',
 'AVB',
 'OWL',
 'CUZ',
 'CSCO',
 'PSX',
 'ESS',
 'SBRA',
 'LPX',
 'DTM',
 'KNX',
 'INGR',
 'IVT',
 'HRL',
 'TSM',
 'MDT',
 'TXN',
 'MPW',
 'HP',
 'OSK',
 'BKR',
 'VTRS',
 'DLR',
 'ALE',
 'CNXC',
 'LHX',
 'OVV',
 'LH',
 'PARA',
 'ADI',
 'PINC',
 'TAP',
 'DGX',
 'ROIC',
 'MOS',
 'CIVI',
 'BABA',
 'ASH',
 'CF',
 'XOM',
 'RYN',
 'LEN',
 'MTG',
 'TOL',
 'GLW',
 'PSA',
 'JNPR',
 'PLD',
 'SWKS',
 'RDN',
 'KRG',
 'TMO',
 'COP',
 'FAST',
 'FANG',
 'DOC',
 'CCF',
 'FOX',
 'AEO',
 'QCOM',
 'HST',
 'LFUS',
 'CTO',
 'JHG',
 'ROP',
 'FR',
 'SHOO',
 'IR',
 'OXY',
 'SEB',
 'ALG',
 'PFE',
 'PTEN',
 'INVH',
 'X',
 'HLNE',
 'CET',
 'WD',
 'KHC',
 'ARCH',
 'VTS',
 'PKG',
 'MUR',
 'JNJ',
 'UNF',
 'J',
 'CR',
 'NUE',
 'JCI',
 'MRO',
 'GRMN',
 'WPC',
 'KRC',
 'AME',
 'MDC',
 'RHI',
 'VMC'

Rank the target stock list by descending dividend yield.

In [3]:
import yfinance as yf

tickers = list(yf.Tickers(ticker_names).tickers.values())

# pprint(tickers[0].info)
cnt = 0
for ticker in tickers:
    try:
        ticker.info['dividendYield']
    except KeyError:
        tickers.remove(ticker)

sorted_tickers = sorted(tickers, key=lambda x: x.info["dividendYield"], reverse=True)
print(sorted_tickers[0].info['dividendYield'])

0.17889999


For each target stock get the PUT option chain with expiry dates more than 3 days but less than 15 days before the ex-div date ["exDividendDate"].

In [5]:
from datetime import datetime
tickers = []

for ticker in sorted_tickers:
    ticker_dates = {}
    ticker_dates["ticker"] = ticker
    ticker_dates["options"] = []
    exDividendDate = datetime.fromtimestamp(ticker.info["exDividendDate"]).date()
    ticker_dates["exDividendDate"] = exDividendDate
    for option in ticker.options:
        date = datetime.strptime(option, '%Y-%m-%d').date() 
        difference = exDividendDate - date
        if 3 < difference.days < 15:
            ticker_dates["options"].append(option)
    if len(ticker_dates["options"]) > 0:
        tickers.append(ticker_dates)

pprint(tickers)

[{'exDividendDate': datetime.date(2024, 1, 24),
  'options': ['2024-01-12', '2024-01-19'],
  'ticker': yfinance.Ticker object <PFE>},
 {'exDividendDate': datetime.date(2024, 2, 28),
  'options': ['2024-02-16'],
  'ticker': yfinance.Ticker object <TSN>},
 {'exDividendDate': datetime.date(2024, 1, 11),
  'options': ['2023-12-29', '2024-01-05'],
  'ticker': yfinance.Ticker object <HRL>},
 {'exDividendDate': datetime.date(2024, 1, 2),
  'options': ['2023-12-29'],
  'ticker': yfinance.Ticker object <CSCO>},
 {'exDividendDate': datetime.date(2024, 1, 15),
  'options': ['2024-01-05'],
  'ticker': yfinance.Ticker object <EOG>},
 {'exDividendDate': datetime.date(2024, 1, 3),
  'options': ['2023-12-29'],
  'ticker': yfinance.Ticker object <AEO>},
 {'exDividendDate': datetime.date(2024, 1, 16),
  'options': ['2024-01-05', '2024-01-12'],
  'ticker': yfinance.Ticker object <ACN>},
 {'exDividendDate': datetime.date(2024, 1, 10),
  'options': ['2023-12-29', '2024-01-05'],
  'ticker': yfinance.Ticker 

Remove all contracts with an open interest less than 1000 contracts.
Remove all contracts with a strike price greater than or less than 2% of the current market price.  